In [1]:
import surprise
from tqdm import tqdm

file_path = 'data/data_train.csv'

# I have to convert the 
def deal_line(line):
    pos, rating = line.split(',')
    row, col = pos.split("_")
    row = row.replace("r", "")
    col = col.replace("c", "")
    return int(row), int(col), float(rating)

def read_txt(path):
    """read text file from path."""
    with open(path, "r") as f:
        return f.read().splitlines()
    
lines = read_txt(file_path)[1:]
data = [deal_line(line) for line in lines]

with open('data/kiru.csv', 'w') as f:
    for item in data:
        f.write("{},{},{}\n".format(item[0], item[1], item[2]))

from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import KNNWithMeans
from surprise.model_selection import cross_validate

# path to dataset file
file_path = 'data/kiru.csv'
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_file(file_path, reader=reader)

#svd = SVD()

svd = KNNWithMeans()
print("call cross_validate")

# We can now use this dataset as we please, e.g. calling cross_validate
abc = cross_validate(svd, data, 
                     n_jobs=10, # use all cpu
                     cv=10, verbose=True, 
                     measures=['RMSE'])

svd.predict(str(1), str(1))

lines = read_txt('data/sample_submission.csv')[1:]
data = [deal_line(line) for line in lines]

predictions = []
for each in tqdm(data):
    pred = svd.predict(str(each[0]), str(each[1])).est
    predictions.append((each[0], each[1], pred))

with open('data/our_pred.csv', 'w') as f:
    f.write("Id,Prediction\n")
    for item in tqdm(predictions):
        f.write("r{}_c{},{}\n".format(item[0], item[1], int(round(item[2]))))